In [1]:
# Import libraries
import sys
import tweepy
import re
import matplotlib.pyplot as plt
import pandas as pd
import unidecode
import math
from collections import Counter

# Authenticate to Twitter API with Tweepy
consumer_key='S7e4bcPSpNhfHEeO64pjylwFC'
consumer_secret='cGLaCjvlDvfwqzkEn7hN3AkZlUMtrhLz7SFczsASnkexy10BBx'
access_key='959019738855862273-5rqxlIGK9aCvlo2z9JWahnFPcpfln2f'
access_secret='z2NQevSQfCdEp67FWYqnYd531J6KMCzp2zGjgESiP1JsX'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True) # Wait if rate limit appears

# Read cities dataset
df_cities = pd.read_csv('cities.csv', encoding='utf-8')

In [ ]:
df_cities.head()

In [2]:
# Load used CAs
Andalusia = ["Andalusia", "37.3399964,-4.5811614,250km"]
Madrid = ["Madrid", "40.5248319,-3.7715628,60km"]
Catalonia = ["Catalonia", "41.8523094,1.5745043,150km"]
Canary_Islands = ["Canary_Islands", "28.5306525,-15.7464439,400km"]
Basque_Country = ["Basque_Country", "42.9911816,-2.5543023,100km"]

# Other CAs
#Extremadura = 
#CastillaLaMancha = 
#CastillaLeon = 
#Cantabria = 
#ComunidadValenciana = 
#Aragon = 
#LaRioja = 
#Navarra = 
#Asturias = 
#Murcia = 

CAS = [Andalusia, Madrid, Catalonia, Basque_Country, Canary_Islands]

In [ ]:
# 1. Get 300 tweets (in case some of them fail to be retrieved) for each CA
# 2. Extract location of tweet. If location doesnt match one of the cities in the CA: ignore. Else:
# 3. Save ID of the tweet's user
CAS_ids = []
for CA in CAS:
    CA_cities = df_cities[df_cities["admin"] == CA[0]]["city"]
    tweets = tweepy.Cursor(api.search, geocode = CA[1]).items(500)  
    CA_ids = []
    for tweet in tweets:
        search_location = re.search('location\':(.+?),', str(tweet.user))
        if search_location:
            found_location = search_location.group(1)
            found_location = unidecode.unidecode(found_location) # Delete accent of location
            if any(word.lower() in found_location.lower() for word in CA_cities): # Check if city matches
                search_id = re.search('id\': (.+?),', str(tweet.user))
                if search_id:
                    found_id = search_id.group(1)
                    CA_ids.append(int(found_id))
                    print(found_id)
    CAS_ids.append(CA_ids)


In [ ]:
# Create dataframe with list of user IDs per each CA
columns  =["Andalusia", "Madrid", "Catalonia", "Basque_Country", "Canary_Islands"]
df_ids = pd.DataFrame(data=list(zip(CAS_ids[0], CAS_ids[1], CAS_ids[2], CAS_ids[3], CAS_ids[4])),columns=columns)
print(df_ids)

In [ ]:
df_ids.to_csv("CAS_ids2.csv", index = False)

In [6]:
df_ids = pd.read_csv("CAS_ids.csv")

In [7]:
df_ids.head()

,Unnamed: 0,Andalusia,Madrid,Catalonia,Basque_Country,Canary_Islands
0,0,3.187391e+09,9.472620e+17,9.962820e+17,5.779776e+08,1.535959e+09
1,1,3.570266e+08,3.091823e+09,1.108110e+18,2.857514e+08,4.040097e+09
2,2,2.847942e+08,3.014721e+09,4.348639e+08,1.976204e+08,3.931719e+09
3,3,1.105070e+18,3.018285e+08,3.781350e+08,4.913358e+08,9.943330e+17
4,4,4.757284e+08,1.005840e+18,9.686820e+17,1.013520e+18,2.361099e+09


In [ ]:
# For each user ID, get 200 tweets and store all of them (some will give error)
tweet_users_text = []
for CA in CAS:
    user_n = 1
    for user_id in df_ids[CA[0]]:
        tweet_n = 1
        user_id = int(user_id)
        user_id = str(user_id)
        try:      
            tweets = tweepy.Cursor(api.user_timeline, id=user_id).items(100)
            for tweet in tweets:
                tweet_text = [user_id, tweet.text, CA[0]]
                if !(math.isnan(tweet.text)):
                    tweet_users_text.append(tweet_text)
                    print("User {} of {} / Tweet {} of {} in {}".format(user_n, len(df_ids[CA[0]]), tweet_n, len(tweets), CA[0]))
                    tweet_n += 1
            user_n += 1
            
        except:
            "Ignore User"   

In [ ]:
# Transform to dataframe
df_tweets = pd.DataFrame(tweet_users_text, columns = ["id", "tweet", "location"])

In [5]:
# Check number of tweets obtained
count_CA = Counter(list(df_tweets["location"])).most_common(5)
print(count_CA)

[('Andalusia', 7237), ('Madrid', 6500), ('Basque_Country', 6300), ('Canary_Islands', 4759), ('Catalonia', 2792)]


In [ ]:
df_tweets.to_csv("tweet_dataset.csv", encoding='utf-8-sig', index = False)

In [2]:
df_tweets = pd.read_csv("tweet_dataset.csv")